In [3]:
from Data.IMDB import IMDB

/home/robin/uu/thesis/hetero-conv-matching/.venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
imdb = IMDB()
original_graph = imdb.load_graph()

In [5]:
original_graph

Graph(num_nodes={'actor': 5257, 'director': 2081, 'movie': 4278},
      num_edges={('actor', 'actortomovie', 'movie'): 12828, ('director', 'directortomovie', 'movie'): 4278, ('movie', 'movietoactor', 'actor'): 12828, ('movie', 'movietodirector', 'director'): 4278},
      metagraph=[('actor', 'movie', 'actortomovie'), ('movie', 'actor', 'movietoactor'), ('movie', 'director', 'movietodirector'), ('director', 'movie', 'directortomovie')])

In [ ]:
num_nearest_init_neighbors_per_type = {"actortomovie": 5, "directortomovie": 5, "movietoactor":5, "movietodirector":5 }
